Hi, so using the methods developed here I was able to download a large dataset of two kinds of images: 1. Shaded relief map tiles of seabed with at least one seamount on it  2. Shaded relief map tiles of seabed without seamounts. With this dataset (that at the moment is 2500 images with seamounts and 1000 without, but could be extended to about 10000 images each) I would like to build a simple neural network with the aim to be able to take on the binary classification task of deciding whether there is a seamount in the image when presented with a new map tile. My first task is to build a baseline model for that, can you assist me?

In [2]:
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

# Parameters
image_folder = "dataset_for_galore_vol_1_1"  # Root directory of the dataset
output_folder = "temp_processed_images"  # Temporary folder for cropped images
image_size = (128, 128)  # Image dimensions for the model
crop_pixels = 70  # Pixels to crop from each border
batch_size = 32
epochs = 15

# Categories
categories = ["with_seamount", "without_seamount"]  # Subfolders in `image_folder`

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Function to crop borders from an image
def crop_fixed_border(image, crop_pixels=5):
    height, width, _ = image.shape
    if height > crop_pixels * 2 and width > crop_pixels * 2:
        cropped_image = image[crop_pixels:height-crop_pixels, crop_pixels:width-crop_pixels]
        return cropped_image
    else:
        return image  # Return original image if cropping not possible

# Preprocess images (cropping and resizing)
print("Preprocessing images...")
for category in categories:
    category_folder = os.path.join(image_folder, category)
    output_category_folder = os.path.join(output_folder, category)
    os.makedirs(output_category_folder, exist_ok=True)
    
    for file_name in os.listdir(category_folder):
        image_path = os.path.join(category_folder, file_name)
        output_path = os.path.join(output_category_folder, file_name)
        try:
            image = cv2.imread(image_path)
            if image is not None:
                cropped_image = crop_fixed_border(image, crop_pixels)
                resized_image = cv2.resize(cropped_image, image_size)
                cv2.imwrite(output_path, resized_image)
        except Exception as e:
            print(f"Error processing {image_path}: {e}")

print("Preprocessing complete.")

# Load processed dataset
def load_data(directory, categories):
    data, labels = [], []
    for label, category in enumerate(categories):
        category_folder = os.path.join(directory, category)
        for file_name in os.listdir(category_folder):
            file_path = os.path.join(category_folder, file_name)
            image = cv2.imread(file_path)
            if image is not None:
                data.append(image)
                labels.append(label)
    return np.array(data), np.array(labels)

data, labels = load_data(output_folder, categories)

# Split into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalize pixel values
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



Preprocessing images...
Preprocessing complete.


/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-30 18:00:26.969070: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Classification report
y_pred = (model.predict(X_test) > 0.5).astype("int32").flatten()
print(classification_report(y_test, y_pred, target_names=categories))

# Save the model
model.save("seamount_classifier_baseline_with_cropping.h5")
print("Model saved.")


Epoch 1/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 23s 281ms/step - accuracy: 0.7202 - loss: 0.5055 - val_accuracy: 0.9719 - val_loss: 0.1126
Epoch 2/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 40s 273ms/step - accuracy: 0.9534 - loss: 0.1543 - val_accuracy: 0.9794 - val_loss: 0.0725
Epoch 3/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 22s 285ms/step - accuracy: 0.9592 - loss: 0.1390 - val_accuracy: 0.9794 - val_loss: 0.0770
Epoch 4/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 40s 271ms/step - accuracy: 0.9795 - loss: 0.0886 - val_accuracy: 0.9195 - val_loss: 0.2157
Epoch 5/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 41s 270ms/step - accuracy: 0.9629 - loss: 0.1130 - val_accuracy: 0.9831 - val_loss: 0.0434
Epoch 6/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 41s 275ms/step - accuracy: 0.9783 - loss: 0.0819 - val_accuracy: 0.9569 - val_loss: 0.1083
Epoch 7/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 22s 277ms/step - accuracy: 0.9673 - loss: 0.0967 - val_accuracy: 0.9738 - val_loss: 0.0763
Epoch 8/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 41s 273ms/step - accuracy: 0.9773 - loss: 0.0707 - val_accu

                  precision    recall  f1-score   support

   with_seamount       0.99      0.99      0.99       392
without_seamount       0.97      0.98      0.98       142

        accuracy                           0.99       534
       macro avg       0.98      0.98      0.98       534
    weighted avg       0.99      0.99      0.99       534

Model saved.
